# CHOLESKY Decomposition/Factorization

Given a symmetric positive definite matrix A, the Cholesky decomposition is an upper triangular matrix U (with strictly positive diagonal entries) such that:

$A=U^TU$

In [ ]:
import pycompss.interactive as ipycompss

In [ ]:
# Start PyCOMPSs runtime with graph and tracing enabled
ipycompss.start(graph=True, monitor=1000)
#trace=True

In [ ]:
from pycompss.api.task import task
from scipy import linalg
from scipy import random
import numpy as np
import ctypes

### Task definitions

In [ ]:
@task(returns=list)
def createBlock(BSIZE, MKLProc, diag):
    import os
    os.environ["MKL_NUM_THREADS"]=str(MKLProc)
    block = np.array(np.random.random((BSIZE, BSIZE)), dtype=np.double,copy=False)
    mb = np.matrix(block, dtype=np.double, copy=False)
    mb = mb + np.transpose(mb)
    if diag:
        mb = mb + 2*BSIZE*np.eye(BSIZE)
    return mb

@task(returns=np.ndarray)
def potrf(A, MKLProc):
    from scipy.linalg.lapack import dpotrf
    import os
    os.environ['MKL_NUM_THREADS']=str(MKLProc)
    A = dpotrf(A, lower=True)[0]
    return A

@task(returns=np.ndarray)
def solve_triangular(A, B, MKLProc):
    from scipy.linalg import solve_triangular
    from numpy import transpose
    import os
    os.environ['MKL_NUM_THREADS']=str(MKLProc)
    B = transpose(B)
    B = solve_triangular(A, B, lower=True)  # , trans='T'
    B = transpose(B)
    return B

@task(returns=np.ndarray)
def gemm(alpha, A, B, C, beta, MKLProc):
    from scipy.linalg.blas import dgemm
    from numpy import transpose
    import os
    os.environ['MKL_NUM_THREADS']=str(MKLProc)
    B = transpose(B)
    C = dgemm(alpha, A, B, c=C, beta=beta)
    return C


### Auxiliar functions

In [ ]:
def genMatrix(MKLProc):
    for i in range(MSIZE):
        A.append([])
        for j in range(MSIZE):
            A[i].append([])
    for i in range(MSIZE):
        mb = createBlock(BSIZE, MKLProc, True)
        A[i][i]=mb
        for j in range(i+1,MSIZE):
            mb = createBlock(BSIZE, MKLProc, False)
            A[i][j]=mb
            A[j][i]=mb

In [ ]:
def cholesky_blocked(A):
    import os
    for k in range(MSIZE):
        # Diagonal block factorization
        A[k][k] = potrf(A[k][k], mkl_threads)
        # Triangular systems
        for i in range(k+1, MSIZE):
            A[i][k] = solve_triangular(A[k][k], A[i][k], mkl_threads)
            A[k][i] = np.zeros((BSIZE,BSIZE))

        # update trailing matrix
        for i in range(k+1, MSIZE):
            for j in range(i, MSIZE):
                A[j][i] = gemm(-1.0, A[j][k], A[i][k], A[j][i], 1.0, mkl_threads)

    return A

## MAIN Code

Parameters (that can be configured in the following cell):
* MSIZE: Matrix size (default: 2)
* BSIZE: Block size (default: 2)
* mkl_threads: Number of MKL threads (default: 1)

In [ ]:
import time
import sys
import os
import copy 
from scipy.linalg.blas import dgemm
from pycompss.api.api import compss_barrier
from pycompss.api.api import compss_wait_on

MSIZE = 8
BSIZE = 1024
mkl_threads = 1
    
# Generate supermatrix
A = []
res = []
genMatrix(mkl_threads)

res = cholesky_blocked(A)
compss_barrier()


In [ ]:
# Plot Result
print "---------- PARAMS ----------"
print "MSIZE:{}".format(MSIZE)
print "BSIZE:{}".format(BSIZE)
print "----------------------------"

In [ ]:
ipycompss.stop()